#### using pandas to get familair with your data

In [2]:
import pandas as pd

In [3]:
melbourne_data = pd.read_csv('./melb_data.csv',parse_dates=True)

In [4]:
melbourne_data.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [5]:
y = melbourne_data.Price

In [6]:
melbourne_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
X = melbourne_data[melbourne_features]

In [7]:
X.isnull().sum()

Rooms         0
Bathroom      0
Landsize      0
Lattitude     0
Longtitude    0
dtype: int64

In [8]:
X.describe()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
count,13580.000000,13580.000000,13580.000000,13580.000000,13580.000000
mean,2.937997,1.534242,558.416127,-37.809203,144.995216
std,0.955748,0.691712,3990.669241,0.079260,0.103916
min,1.000000,0.000000,0.000000,-38.182550,144.431810
25%,2.000000,1.000000,177.000000,-37.856822,144.929600
50%,3.000000,1.000000,440.000000,-37.802355,145.000100
75%,3.000000,2.000000,651.000000,-37.756400,145.058305
max,10.000000,8.000000,433014.000000,-37.408530,145.526350


In [9]:
X.dtypes

Rooms           int64
Bathroom      float64
Landsize      float64
Lattitude     float64
Longtitude    float64
dtype: object

In [10]:
X.head()

,Rooms,Bathroom,Landsize,Lattitude,Longtitude
0,2,1.0,202.0,-37.7996,144.9984
1,2,1.0,156.0,-37.8079,144.9934
2,3,2.0,134.0,-37.8093,144.9944
3,3,2.0,94.0,-37.7969,144.9969
4,4,1.0,120.0,-37.8072,144.9941


#### Desition tree for regreesion:

In [11]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)

melbourne_model = DecisionTreeRegressor(random_state=1)

melbourne_model.fit(X_train,y_train)

DecisionTreeRegressor(random_state=1)

In [12]:

y_predict = melbourne_model.predict(X_test)

error = mean_absolute_error(y_test,y_predict)

print(f'the mean absolute error is : {error}')

the mean absolute error is : 250429.6127638684


In [43]:
y.mean()

1075684.079455081

In [13]:
def get_MAE_of_model(model,X_train,y_train,X_test,y_test) -> None:
    model.fit(X_train,y_train)
    y_test_predict = model.predict(X_test)
    return mean_absolute_error(y_test,y_test_predict)


In [34]:
from tqdm import tqdm
min_error = float('inf')
min_leafs = None
for max_leafs_nodes in tqdm(range(5,1000)):
    error = get_MAE_of_model(DecisionTreeRegressor(max_leaf_nodes=max_leafs_nodes,random_state=0),X_train,y_train,X_test,y_test)
    if min_error >= error:
        min_error = error
        min_leafs = max_leafs_nodes

print(f'the best number of leafs {min_leafs} and the error {error}')

100%|██████████| 995/995 [00:35<00:00, 27.89it/s]

the best number of leafs 480 and the error 237549.01895591972


In [35]:
model_for_deploment = DecisionTreeRegressor(max_leaf_nodes=480,random_state=0)
model_for_deploment.fit(X,y)

DecisionTreeRegressor(max_leaf_nodes=480, random_state=0)

In [36]:
y_predict = model_for_deploment.predict(X)

mean_absolute_error(y,y_predict)

158616.13807307693

#### Random Forest for regression:

In [37]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model=  RandomForestRegressor()

forest_model.fit(X,y)

mean_absolute_error(y,forest_model.predict(X))

66892.79960507102

In [40]:
f'random forest improve the model by {(158616.13807307693-66892.79960507102)/158616.13807307693:_.3f}'

'random forest improve the model by 0.578'

#### Missing values

In [58]:
numericale_columns = melbourne_data.select_dtypes(['float','int']).columns

In [59]:
melbourne_data.isnull().sum()

Suburb              0
Address             0
Rooms               0
Type                0
Price               0
Method              0
SellerG             0
Date                0
Distance            0
Postcode            0
Bedroom2            0
Bathroom            0
Car                62
Landsize            0
BuildingArea     6450
YearBuilt        5375
CouncilArea      1369
Lattitude           0
Longtitude          0
Regionname          0
Propertycount       0
dtype: int64

In [60]:
X = melbourne_data[numericale_columns]
y = X.pop('Price')
X_train,X_test,y_train,y_test = train_test_split(X,y)

#### the first aproach is drop the columns witch contains missing values:

In [62]:
cols_with_missing = [ col for col in X_train.columns if X_train[col].isnull().any()]

In [63]:
cols_with_missing

['Car', 'BuildingArea', 'YearBuilt']

In [64]:
reduced_X_train = X_train.drop(cols_with_missing,axis=1)
reduced_X_test = X_test.drop(cols_with_missing,axis=1)

In [65]:
get_MAE_of_model(RandomForestRegressor(n_estimators=10,random_state=0),reduced_X_train,y_train,reduced_X_test,y_test)

190203.30046847608

##### the seconde aproach is imputation:

In [70]:
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_test = pd.DataFrame(my_imputer.transform(X_test))

### change nomatio of the columns
imputed_X_train.columns = X_train.columns
imputed_X_test.columns = X_test.columns

In [72]:
get_MAE_of_model(RandomForestRegressor(n_estimators=10,random_state=0),imputed_X_train,y_train,imputed_X_test,y_test)

185828.2988140823

##### the third aproch :

In [73]:
X_train_plus = X_train.copy()
X_test_plus = X_test.copy()

for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull().astype('int')
    X_test_plus[col + '_was_missing'] = X_test_plus[col].isnull().astype('int')

my_imputer_plus = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer_plus.fit_transform(X_train_plus))
imputed_X_test_plus = pd.DataFrame(my_imputer_plus.transform(X_test_plus))

### change nomatio of the columns
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_test_plus.columns = X_test_plus.columns

In [75]:
get_MAE_of_model(RandomForestRegressor(n_estimators=10,random_state=0),imputed_X_train_plus,y_train,imputed_X_test_plus,y_test)

187580.2144350936

#### Categorical variables:

In [14]:
y = melbourne_data.Price
X = melbourne_data.drop('Price',axis=1)

In [15]:
X_train,X_valid,y_train,y_valid = train_test_split(X,y,test_size=0.2)

In [16]:
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
X_train_dropted = X_train.drop(cols_with_missing,axis=1)
X_valid_dropted = X_valid.drop(cols_with_missing,axis=1)

low_categorical_cols = [col for col in X_train_dropted.columns  if X_train_dropted[col].dtype == 'object' and X_train_dropted[col].nunique() < 10]

In [17]:
numericale_columns  = X_train_dropted.select_dtypes(['int','float']).columns

In [18]:
my_cols = list(low_categorical_cols) + list(numericale_columns)
X_train =  X_train_dropted[my_cols]
X_valid =  X_valid_dropted[my_cols]

In [19]:
list(X_train.select_dtypes(['object']).columns)

['Type', 'Method', 'Regionname']

##### approach 1 drop the categorical features:

In [20]:
from sklearn.ensemble import RandomForestRegressor
X_train_dropted = X_train.copy().drop(['Type', 'Method', 'Regionname'],axis=1)
X_valid_dropted = X_valid.copy().drop(['Type', 'Method', 'Regionname'],axis=1)

get_MAE_of_model(RandomForestRegressor(n_estimators=100,random_state=0),X_train_dropted,y_train,X_valid_dropted,y_valid)

183058.587774242

##### approach 2 ordinal encoding:

In [21]:
from sklearn.preprocessing import OrdinalEncoder
for_encode = ['Type', 'Method', 'Regionname']

X_train_ordinal_encoder = X_train.copy()
X_valid_ordinal_encoder = X_valid.copy()
my_encoder = OrdinalEncoder()

X_train_ordinal_encoder[for_encode] = my_encoder.fit_transform(X_train[for_encode])
X_valid_ordinal_encoder[for_encode] = my_encoder.fit_transform(X_valid[for_encode])

In [22]:
get_MAE_of_model(RandomForestRegressor(n_estimators=100,random_state=0),X_train_ordinal_encoder,y_train,X_valid_ordinal_encoder,y_valid)

169132.52218160458

##### approach 3 one hot encoding

In [46]:
from sklearn.preprocessing import OneHotEncoder

one_hot_encoding = OneHotEncoder(handle_unknown='ignore',sparse=False)

OH__train  = pd.DataFrame(one_hot_encoding.fit_transform(X_train[for_encode]))
OH__valid = pd.DataFrame(one_hot_encoding.transform(X_valid[for_encode]))

OH__train.index = X_train.index
OH__valid.index = X_valid.index

num_X_train = X_train.drop(for_encode,axis=1)
num_X_valid = X_valid.drop(for_encode,axis=1)

final_X_train = pd.concat([num_X_train,OH__train],axis=1)
final_X_valid  = pd.concat([num_X_valid,OH__valid],axis=1)


In [47]:
final_X_train.columns

Index([        'Rooms',      'Distance',      'Postcode',      'Bedroom2',
            'Bathroom',      'Landsize',     'Lattitude',    'Longtitude',
       'Propertycount',               0,               1,               2,
                     3,               4,               5,               6,
                     7,               8,               9,              10,
                    11,              12,              13,              14,
                    15],
      dtype='object')

In [48]:
final_X_train.dtypes

Rooms              int64
Distance         float64
Postcode         float64
Bedroom2         float64
Bathroom         float64
Landsize         float64
Lattitude        float64
Longtitude       float64
Propertycount    float64
0                float64
1                float64
2                float64
3                float64
4                float64
5                float64
6                float64
7                float64
8                float64
9                float64
10               float64
11               float64
12               float64
13               float64
14               float64
15               float64
dtype: object

In [49]:
get_MAE_of_model(RandomForestRegressor(n_estimators=100,random_state=0),final_X_train,y_train,final_X_valid,y_valid)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


170063.69194193144

### pipe lines:

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
data = pd.read_csv('./melb_data.csv')

# Separate target from predictors
y = data.Price
X = data.drop(['Price'], axis=1)

# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [17]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cateee', categorical_transformer, categorical_cols)
    ])

In [18]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

In [19]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)

MAE: 160679.18917034855


### cross validation:

In [20]:

data = pd.read_csv('./melb_data.csv')

# Select subset of predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X = data[cols_to_use]

# Select target
y = data.Price

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

my_pipeline = Pipeline(steps=[('preprocessor', SimpleImputer()),
                              ('model', RandomForestRegressor(n_estimators=50,
                                                              random_state=0))
                             ])

In [22]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

MAE scores:
 [301628.7893587  303164.4782723  287298.331666   236061.84754543
 260383.45111427]


In [23]:
print("Average MAE score (across experiments):")
print(scores.mean())

Average MAE score (across experiments):
277707.3795913405


### XGboost

In [3]:
from tqdm import notebook
import time as tm
for i in notebook.tqdm(range(10)):
    tm.sleep(0.5)

  0%|          | 0/10 [00:00<?, ?it/s]